#Setup

In [ ]:
import numpy as np
from skimage.measure import regionprops, label
from skimage.io import imread
import os
import tensorflow as tf
import pickle
import time
from skimage.color import rgb2hsv, rgb2gray
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model_segmentation = tf.keras.models.load_model('model_segmentation_depthwiseConv2D.h5', compile=False)
model_color_filter2 = tf.keras.models.load_model('model_color_filter_2.h5', compile=False)
model_color_filter3 = tf.keras.models.load_model('model_color_filter_3.h5', compile=False)
model_shape = pickle.load(open('model_shape_filter.sav', 'rb'))
crop = np.ones((40,40,3))*255

path_predictions = 'predictions/'
os.mkdir(path_predictions)

# Define functions

In [ ]:
def loadImages(path, subfolder):
    image_files = sorted([os.path.join(path, subfolder, file)
                          for file in os.listdir(path + "/" + subfolder)
                          if file.endswith('.bmp')])
    return image_files

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask

def prediction(img_mosaico):
    image = img_mosaico/255
    m, n, c = image.shape
    image = np.array(image).reshape(1, m, n, c)
    pred_mask = model_segmentation(image, training=False)
    Y_new = np.asarray(create_mask(pred_mask[0]), dtype='uint8')
    Y_new = np.array(Y_new).reshape(400, 400, 1)
    return Y_new

In [ ]:
def segmentation_images(im):
    img = imread(im)
    m, n, p = img.shape
    img_shape_mosaicos = np.ones((m + 160, n + 212, p), dtype=np.uint8)
    img_shape_mosaicos.fill(255)

    img_shape_mosaicos[0:m, 0:n, :] = img

    predict = np.zeros([m + 160, n + 212, p], dtype=np.uint8)

    for i in range(3):
        for j in range(4):
            img_mosaico = img_shape_mosaicos[400 * i:400 * (i + 1), 400 * j:400 * (j + 1)]
            Y_new = prediction(img_mosaico)
            Y_new = np.uint8(Y_new * 255)
            Y_new = np.dstack([Y_new, Y_new, Y_new])
            predict[400 * i:400 * (i + 1), 400 * j:400 * (j + 1), :] = Y_new

    return predict

In [ ]:
def component_h(img_h):
    size_m, size_n = img_h.shape
    c = 0
    for sm in range(size_m):
        for sn in range(size_n):
            if img_h[sm, sn] >= 0.74:
                c = c + 1
    c = (c * 100) / (size_m * size_n)
    if c >= 50:
        return False, c
    else:
        return True, c

In [ ]:
def count_bacillus(images_diag, bacilos_original):
    bacilos_pred = 0
    preds = []
    total_area = 0
    total_cor = 0
    total_contexto = 0
    total_shape = 0

    for i in range(len(images_diag)):
        time_start_total = time.time()

        img_rgb_ori = imread(images_diag[i])
        img_pred = segmentation_images(images_diag[i])

        m, n, p = img_rgb_ori.shape
        img_rgb = np.ones((m + 160, n + 212, p), dtype=np.uint8)
        img_rgb.fill(255)

        img_rgb[0:m, 0:n, :] = img_rgb_ori

        label_pred, n_pred = label(img_pred, connectivity=2, return_num=True)
        regions_pred = regionprops(label_pred)
        label_pred_gray, n_pred_gray = label(rgb2gray(img_pred), connectivity=2, return_num=True)
        regions_pred_gray = regionprops(label_pred_gray)

        if (n_pred >= 1):
            n_pred = 0
            for props, props_gray in zip(regions_pred, regions_pred_gray):

                df_shape = pd.DataFrame([])

                if 25 < props.area < 900:
                    total_area = total_area + 1

                    y0, x0, z0 = props.centroid
                    y0, x0 = int(y0), int(x0)

                    crop_zeros = np.zeros([40, 40, 3], dtype=np.uint8)

                    if y0 - 20 >= 0 and y0 + 20 <= img_rgb.shape[0] and x0 - 20 >= 0 and x0 + 20 <= img_rgb.shape[1]:
                        part_img_rgb = img_rgb[y0 - 20:y0 + 20, x0 - 20:x0 + 20, :]
                        part_img_pred = img_pred[y0 - 20:y0 + 20, x0 - 20:x0 + 20, :]

                    if y0 - 20 < 0 and y0 + 20 <= img_rgb.shape[0] and x0 - 20 >= 0 and x0 + 20 <= img_rgb.shape[1]:
                        part_img_rgb = crop.copy()
                        part_img_pred = crop_zeros.copy()
                        sub = y0 - 20
                        value = 20 + sub
                        part_img_rgb[20 - value:40, 0:40, :] = img_rgb[y0 - value:y0 + 20, x0 - 20:x0 + 20, :]
                        part_img_pred[20 - value:40, 0:40, :] = img_pred[y0 - value:y0 + 20, x0 - 20:x0 + 20, :]

                    if y0 - 20 >= 0 and y0 + 20 > img_rgb.shape[0] and x0 - 20 >= 0 and x0 + 20 <= img_rgb.shape[1]:
                        part_img_rgb = crop.copy()
                        part_img_pred = crop_zeros.copy()
                        som = y0 + 20
                        sub = som - img_rgb.shape[0]
                        value = 20 - sub
                        part_img_rgb[0:20 + value, 0:40, :] = img_rgb[y0 - 20:y0 + value, x0 - 20:x0 + 20, :]
                        part_img_pred[0:20 + value, 0:40, :] = img_pred[y0 - 20:y0 + value, x0 - 20:x0 + 20, :]

                    if y0 - 20 >= 0 and y0 + 20 <= img_rgb.shape[0] and x0 - 20 < 0 and x0 + 20 <= img_rgb.shape[1]:
                        part_img_rgb = crop.copy()
                        part_img_pred = crop_zeros.copy()
                        sub = x0 - 20
                        value = 20 + sub
                        part_img_rgb[0:40, 20 - value:40, :] = img_rgb[y0 - 20:y0 + 20, x0 - value:x0 + 20, :]
                        part_img_pred[0:40, 20 - value:40, :] = img_pred[y0 - 20:y0 + 20, x0 - value:x0 + 20, :]

                    if y0 - 20 >= 0 and y0 + 20 <= img_rgb.shape[0] and x0 - 20 >= 0 and x0 + 20 > img_rgb.shape[1]:
                        part_img_rgb = crop.copy()
                        part_img_pred = crop_zeros.copy()
                        som = x0 + 20
                        sub = som - img_rgb.shape[1]
                        value = 20 - sub
                        part_img_rgb[0:40, 0:20 + value, :] = img_rgb[y0 - 20:y0 + 20, x0 - 20:x0 + value, :]
                        part_img_pred[0:40, 0:20 + value, :] = img_pred[y0 - 20:y0 + 20, x0 - 20:x0 + value, :]

                    img_hsv = rgb2hsv(part_img_rgb)

                    isValid, pc = component_h(img_hsv[:, :, 0])

                    area = props_gray.area
                    maior_eixo = props_gray.major_axis_length
                    menor_eixo = props_gray.minor_axis_length
                    excentricidade = props_gray.eccentricity
                    momento_hu = props_gray.moments_hu

                    df_shape = df_shape.append([[area, maior_eixo, menor_eixo, excentricidade, momento_hu[0],
                                                 momento_hu[1], momento_hu[2], momento_hu[3], momento_hu[4],
                                                 momento_hu[5], momento_hu[6]]], ignore_index=True)

                    if isValid:
                        part_img_rgb1 = part_img_rgb / 255

                        patch = np.dstack([(part_img_rgb1[:, :, 0] - part_img_rgb1[:, :, 1]),
                                           (part_img_rgb1[:, :, 0] - part_img_rgb1[:, :, 2]),
                                           img_hsv[:, :, 0], img_hsv[:, :, 1], img_hsv[:, :, 2]])
                        form = np.array(patch).reshape((1, 40, 40, 5))
                        p2 = model_color_filter2(form, training=False)

                        if p2 > 0.5:
                            total_cor = total_cor + 1
                            form = np.array(part_img_rgb/255).reshape((1, 40, 40, 3))
                            p3 = model_color_filter3(form, training=False)

                            if p3 > 0.5:
                                total_contexto = total_contexto + 1

                                p_shape = model_shape.predict(df_shape)
                                if p_shape > 0.5:
                                    total_shape = total_shape + 1
                                    n_pred = n_pred + 1

        bacilos_pred = bacilos_pred + n_pred
        print("Tempo Total da imagem: " + str(time.time() - time_start_total))

        preds.append(n_pred)

    print('Qtd Bacilos Original: ' + str(bacilos_original))
    print('Qtd Bacilos Preditos: ' + str(bacilos_pred) + '\n')

    print("Total Area: " + str(total_area))
    print("Total Cor: " + str(total_cor))
    print("Total Contexto: " + str(total_contexto))
    print("Total Shape: " + str(total_shape))

    return bacilos_pred

# Count bacilli

In [ ]:
list_diagnoses_gt = ['NEGATIVE', 'SCANTY', '1+', '2+', '3+']

for d in range(0, 5):
    path_diag = "DDS4/" + list_diagnoses_gt[d]

    fi = open('predictions/predictions_' + list_diagnoses_gt[d] + '.txt', 'w')
    fi.close()

    for conj in range(0,50):
        time_start_conj = time.time()
        print('\n SET' + str(conj))

        path_diag_conj = 'SET' + str(conj)

        images_diag = loadImages(path_diag, path_diag_conj)

        f = open(path_diag + "/" + path_diag_conj + "/ground_truth_" + str(conj) + ".txt", 'r')
        lines = f.read().splitlines()
        bacilos_original = lines[0]

        path_save = path_diag_seg + '/' + path_diag_conj
        bacilos_pred = count_bacillus(images_diag, bacilos_original)

        f = open('predictions/predictions_' + list_diagnoses_gt[d] + '.txt', "a")
        f.write(str(bacilos_pred) + '\n')

        print("Tempo Total Conjunto: " + str(time.time() - time_start_conj))